<a href="https://colab.research.google.com/github/AkhilVinayakp/HAR_LSTM_CNN/blob/main/LSmt_cnn_UCIHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
"""
contains from raw data preprcessing to model creation
preprocessing
  1.linear interpolation
  2.MinMaxScaling

Segmentation
  1.windowing
Model Training
  1.lstm cnn
  
  freezed
"""

'\ncontains from raw data preprcessing to model creation\npreprocessing\n  1.linear interpolation\n  2.MinMaxScaling\n\nSegmentation\n  1.windowing\nModel Training\n  1.lstm cnn\n  \n  freezed\n'

In [118]:
# imports
import tensorflow as tf
import requests
import pandas as pd
import numpy as np
from zipfile import ZipFile

In [119]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip'
r = requests.get(url=url, allow_redirects = False)
open('uci.zip','wb').write(r.content)

60999314

In [120]:
!ls

activity_labels.txt  features.txt  test   uci.zip
features_info.txt    README.txt    train


In [121]:
with ZipFile('uci.zip') as zip:
  zip.extractall()

In [122]:
!ls

 activity_labels.txt   features.txt   README.txt   train	      uci.zip
 features_info.txt     __MACOSX       test	  'UCI HAR Dataset'


In [123]:
%cd 'UCI HAR Dataset'

/content/UCI HAR Dataset/UCI HAR Dataset/UCI HAR Dataset


In [124]:
!ls

activity_labels.txt  features_info.txt	features.txt  README.txt  test	train


In [125]:
# reading available for 3 axis seperatly adding function to load each as dataframe
def load_axis_data(path):
  df = pd.read_csv(path, header = None, delim_whitespace=True)
  return df

In [126]:
df_X = load_axis_data('train/Inertial Signals/total_acc_x_train.txt')

In [127]:
df_y = load_axis_data('train/Inertial Signals/total_acc_y_train.txt')
df_z = load_axis_data('train/Inertial Signals/total_acc_z_train.txt')


In [128]:
# printing the shape of each 
print(f"shape of x axis {df_X.shape}")
print(f"shape of y axis {df_y.shape}")
print(f"shape of z axis {df_z.shape}")

shape of x axis (7352, 128)
shape of y axis (7352, 128)
shape of z axis (7352, 128)


In [129]:
# stacking each values to one variable
st_df = np.dstack([df_X,df_y,df_z])

In [130]:
st_df.shape

(7352, 128, 3)

In [131]:
st_df.dtype

dtype('float64')

In [132]:
type(st_df)

numpy.ndarray

In [161]:
# loading the train labels 
df_train_labels = load_axis_data('train/y_train.txt')
df_train_labels[0] = df_train_labels[0].astype(np.int64)

In [162]:
df_train_labels.shape

(7352, 1)

In [163]:
df_train_labels.dtypes

0    int64
dtype: object

In [164]:
df_train_labels.head()

,0
0,5
1,5
2,5
3,5
4,5


In [137]:
# from sklearn.preprocessing import OneHotEncoder

In [170]:
# enc = OneHotEncoder()
# df_train_labels =  enc.fit_transform(df_train_labels)
df_train_labels = tf.one_hot(df_train_labels, depth=6)

In [171]:
df_train_labels.shape

TensorShape([7352, 1, 6])

In [172]:
df_train_labels.dtype

tf.float32

In [141]:
df_X_test = load_axis_data('test/Inertial Signals/total_acc_x_test.txt')
df_y_test = load_axis_data('test/Inertial Signals/total_acc_y_test.txt')
df_z_test = load_axis_data('test/Inertial Signals/total_acc_z_test.txt')
st_df_test = np.dstack([df_X_test,df_y_test,df_z_test])

In [142]:
st_df_test.shape

(2947, 128, 3)

In [174]:
df_test_labels = load_axis_data('test/y_test.txt')
df_test_labels[0] = df_test_labels[0].astype(np.int64)
df_test_labels.dtypes

0    int64
dtype: object

In [175]:
df_test_labels =  tf.one_hot(df_test_labels, depth=6)

In [176]:
df_test_labels.dtype

tf.float32

In [177]:
df_test_labels.shape

TensorShape([2947, 1, 6])

In [178]:
df_test_labels.dtype

tf.float32

In [148]:
# model creation and testing 

In [149]:
# # adding globel average pooling to the model
# codding lstm layers and cnn
model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: x,
                      input_shape=[128, 3]),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.LSTM(32, return_sequences=True),
  # tf.keras.layers.SpatialDropout1D(0.2),
  # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  # tf.keras.layers.Reshape((128,64,1)),
  tf.keras.layers.Conv1D(64,5,strides=2),
  tf.keras.layers.MaxPooling1D(2, strides = 2),
  # tf.keras.layers.Dropout(.4),
  tf.keras.layers.Conv1D(128,3,strides=1),  
  # tf.keras.layers.MaxPooling2D((2, 2), strides = (2,2)),
  # tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1)),
  # parameters ?
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(6, activation='softmax')
  
])

In [150]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 128, 3)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128, 32)           4608      
_________________________________________________________________
lstm_7 (LSTM)                (None, 128, 32)           8320      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 62, 64)            10304     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 31, 64)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 29, 128)           24704     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)              

In [179]:
# designing class back to save the best model
from tensorflow.keras.callbacks import ModelCheckpoint
save = ModelCheckpoint('best_model_uci_n3.h5', save_best_only = True)

In [186]:
print(st_df.dtype)
print(st_df.shape)

TensorShape([7352, 128, 3])

In [181]:
# st_df = tf.cast(st_df, dtype=tf.float32)

In [187]:
# st_df_test = tf.cast(st_df_test, dtype=tf.float32)
print(st_df_test.dtype)
print(st_df_test.shape)

<dtype: 'float32'>
(2947, 128, 3)


In [191]:
# df_train_labels= tf.cast(df_train_labels, dtype=tf.float32)
print(df_train_labels.dtype)
print(df_train_labels.shape)
df_train_labels = tf.reshape(df_train_labels,[7352,6])
print(df_train_labels.shape)

<dtype: 'float32'>
(7352, 1, 6)
(7352, 6)


In [193]:
print(df_test_labels.dtype)
print(df_test_labels.shape)
df_test_labels = tf.reshape(df_test_labels,[2947,6])
print(df_test_labels.shape)

<dtype: 'float32'>
(2947, 1, 6)
(2947, 6)


In [194]:
# with out bidirection
optimizer = tf.keras.optimizers.Adam(0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics = ['accuracy'] )

history = model.fit(x = st_df,y = df_train_labels,validation_data=(st_df_test, df_test_labels), epochs=200,callbacks=[save], verbose=1, batch_size=192) 

Epoch 1/200
39/39 [==============================] - 36s 40ms/step - loss: 1.2698 - accuracy: 0.2435 - val_loss: 1.2775 - val_accuracy: 0.1897
Epoch 2/200
39/39 [==============================] - 1s 17ms/step - loss: 1.3222 - accuracy: 0.2424 - val_loss: 1.6721 - val_accuracy: 0.2548
Epoch 3/200
39/39 [==============================] - 1s 16ms/step - loss: 1.9559 - accuracy: 0.2542 - val_loss: 2.4671 - val_accuracy: 0.1666
Epoch 4/200
39/39 [==============================] - 1s 17ms/step - loss: 3.3294 - accuracy: 0.2033 - val_loss: 1.6086 - val_accuracy: 0.1323
Epoch 5/200
39/39 [==============================] - 1s 16ms/step - loss: 2.6408 - accuracy: 0.2106 - val_loss: 1.7454 - val_accuracy: 0.2464
Epoch 6/200
39/39 [==============================] - 1s 16ms/step - loss: 3.4387 - accuracy: 0.2002 - val_loss: 2.6195 - val_accuracy: 0.0842
Epoch 7/200
39/39 [==============================] - 1s 16ms/step - loss: 5.7972 - accuracy: 0.1717 - val_loss: 4.4367 - val_accuracy: 0.3210
Epoch

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
m_acc = history.history['accuracy']
fig, ax = plt.subplots(figsize = (15,5))
ax.plot(m_acc, color = 'r')
ax.plot(history.history['val_accuracy'], color = 'b')
# plt.legend()

In [ ]:
loss = history.history['loss']
fig, ax = plt.subplots(figsize = (15,5))
ax.plot(loss, color = 'r')
ax.plot(history.history['val_loss'], color = 'b')